In [1]:
using NPZ
using CSV
using FFTW
using HDF5
using Optim
using PyPlot
using PyCall
using DataFrames
using Statistics
using Distributed
using OffsetArrays
using SparseArrays
using StaticArrays
using SharedArrays
using LinearAlgebra
using ProgressMeter
using BenchmarkTools
using DelimitedFiles
using ImageFiltering
using DelimitedFiles

#
cmasher = pyimport("cmasher")

# include custom code
include("./Test/runtests.jl")
include("./Source/ActivePolymer.jl")
using .ActivePolymer
using .ActivePolymer.CorrelationMatrices

Test Summary:                             | Pass  Total
Validation of transforms.                 |    7      7
  Forward transforms vs analytic results. |    3      3
  Inverse transforms vs input values.     |    4      4


# Notebook description

The purpose of this notebook is to predict contact probability density maps, provided that we know the mechanical properties of the polymer.

In [2]:
folder = "predictions"
mkpath(folder)

R, ΔR = ActivePolymer.Optimization.Interface.load_data("Deq1");

# Three-parameter fits
modeltype   = ActivePolymer.Optimization.Model.Full
jacmodule   = ActivePolymer.Jacobian.Discrete
n = 3
parameters  = ActivePolymer.Optimization.Interface.fit_mechanics(
    ΔR, modeltype=modeltype, jacmodule=jacmodule, n=n, padding=0.85)
jacobian    = jacmodule.J(parameters.minimizer[2:end]..., n)

data_groundtruth = npzread("../Share/ABidentities_blobel2021_chr2_35Mb_60Mb.npy");

In [3]:
for difference in 0:0.025:1
    activity = parameters.minimizer[1] .* (1.0 .+ difference .* (data_groundtruth .- 0.5))
    correlation = ActivePolymer.Transform.Forward.compute_conformation(activity, J=jacobian, fourier_type=ActivePolymer.Methods.FastFourier.DCT) 
    separation  = correlation |> ActivePolymer.Methods.Real.correlation_to_separation;
    probability = ((2pi/3)*separation).^(-3/2) |> real
    writedlm(
        [folder, "/contact_probability-activity_difference_", string(difference), ".csv"] |> join,  
        probability, ',')
end